# Comparing and verifying admin level names for adding pcodes

This document reviews how pcodes can be added to .geojson files by fuzzy matching admin names. Sometimes admin names go by different spellings or have errors. With this process we review matches and attach pcodes between the spatial dataset and pcode dataset even if they are slightly different.

This is meant to be a **rough guide** to use as a reference. Currently, it is not detailed enough to work as a full tutorial.

## 1. The context

When creating Pcodes for Brazil, we used codes from the Government of Brazil to label different admin labels. This is in line with best practice, since pcodes will align easier with government. Unfortunately, the shapefiles found on HDX had different IDs. The place names between the government list and the list of HDX also had differences. This made it difficult to use these shapefiles in different services like HXL.

By fuzzy matching place names from the pcode/government dataset to HDX shapefile dataset, we can attach the pcodes to the shapefile from HDX for use in the future.

### 1.1 Using .geojson

We work with a .geojson file in this project. If you currently have a shapefile you can utilise a program, such as QGIS, to export your shapefile layer to a .geojson format.

### 1.2 Required packages
This code uses Python. Please sure the following Python packages are installed:

*   pandas
*   numpy
*   geopandas
*   json
*   fuzzywuzzy 

We'll see use pandas for basic data frame manuvering, geopandas and json for importing and exporting our .geojson file, and fuzzywuzzy for the 


In [ ]:
# Do not worry about installing this a second time if you already installed it.
# I wrote this in Google Colab, so I have to recreate the environment.
pip install geopandas

     |████████████████████████████████| 931kB 4.8MB/s 
     |████████████████████████████████| 14.7MB 312kB/s 
     |████████████████████████████████| 10.4MB 41.0MB/s 


In [ ]:
# Do not worry about installing this a second time if you already installed it.
# I wrote this in Google Colab, so I have to recreate the environment.
pip install fuzzywuzzy

## 2. Import your libraries

First we'll important our libraries, which are pandas, numpy, geopandas, json, and fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 3. Import Your Datasets

I have two datasets. First, a .geojson which I created by exporting the shapefile layer found on HDX. Second, the government dataset (.csv) was taken from the Brazil government website, then pcodes were generated based on their existing IDs.

I am importing these via my Google Drive, but import your dataset however you need to.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Geopandas reads our .geojson file. In the paranthesis put the path to your files.

In [ ]:
brazil_set = gpd.read_file('/content/drive/My Drive/Work/BRC/Brazil Pcode/brazil_adm2.geojson')

Pandas reads our .csv file.

In [ ]:
pcode_adm1 = pd.read_csv('/content/drive/My Drive/Work/BRC/Brazil Pcode/pcode_adm1.csv')

I usually use the .head() command just to check the dataset. We can use this command to look at our two datasets.

In [ ]:
brazil_set.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
0,33,BRA,Brazil,1,Acre,1,Acrelândia,None,0,None,Município,Municipality,None,None,"MULTIPOLYGON (((-67.10586 -9.68811, -67.05984 ..."
1,33,BRA,Brazil,1,Acre,2,Assis Brazil,None,0,None,Município,Municipality,None,None,"MULTIPOLYGON (((-69.66749 -10.96098, -69.66760..."
2,33,BRA,Brazil,1,Acre,3,Brasiléia,None,0,None,Município,Municipality,None,None,"MULTIPOLYGON (((-68.99985 -11.00668, -69.00039..."
3,33,BRA,Brazil,1,Acre,4,Bujari,None,0,None,Município,Municipality,None,None,"MULTIPOLYGON (((-67.94034 -9.33722, -67.94411 ..."
4,33,BRA,Brazil,1,Acre,5,Capixaba,None,0,None,Município,Municipality,None,None,"MULTIPOLYGON (((-68.03945 -10.35492, -68.03228..."


In [ ]:
pcode_adm1.head()

,ISO2,Nome_UF,UF,pcode_adm1
0,BR,Rondônia,11,BR11
1,BR,Acre,12,BR12
2,BR,Amazonas,13,BR13
3,BR,Roraima,14,BR14
4,BR,Pará,15,BR15


We can see from the datasets that the two place name columns we want to compare are "Name_1" and "Nome_UF".

## 4. The fuzzy matching model using FuzzyWuzzy

Our goal is to create a "matches" column in our .geojson dataset that will align with the pcode dataset. Afterwards, we can join the tables using the "matches" column to pull the pcodes into the .geojson file.

First we want to prepare our datasets for FuzzyWuzzy. Since we'll eventually want to carry data FROM the pcode dataset TO the Brazil .geojson file we are making the Brazil dataset as df_1 (as the left table of our join) and the pcode dataset as df_2 (as the right table of our join). 

Next we have to define the "keys." These are the two columns we are performing the match on. We take our first key from our first dataset (the column is called 'Name_1') and we'll take our second key from our second dataset (the column is called 'Nome_UF').

In [ ]:
#Preparing fuzzywuzzy modeling here
#df_1 is the left table to join
#df_2 is the right table to join

df_1 = brazil_set
df_2 = pcode_adm1

#key1 is the key column of the left table
#key2 is the key column of the right table

key1 = brazil_set['NAME_1']
key2 = pcode_adm1['Nome_UF']

Below is the code we'll run. 

On the first line you'll see us setting the parameters of our model. We set the two datasets we're using (df_1 & df_2), their corresponding keys (key1 & key2). 

### 4.1 Threshold

Next we set the threshold. You can see I put threshold=95. This is telling FuzzyWuzzy how "flexible" differences can be. In this context the place names will generally be the close to each other. Because of this, I set it quite high at 90 (the high, the closer the matches need to be).

There are many beginner tutorials that review this part of FuzzyWuzzy (such as here: https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/) if you want to learn more.

### 4.2 Limit

You can see I put limit=2. If we write limit=1 it will take the highest scoring match and put in the column. So why did I put 2? If there is more than one match, we'll want to know about it! More than one match flags we need to verify the match and place name.




In [ ]:
def fuzzy_match(df_1, df_2, key1, key2, threshold=95, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    #Only put this in to create a matches column in the right table. Should not be needed.
    #df_2['matches'] = m2

    return df_1

## 5. Running the code

Now that we've define out code, we run it. Easy enough!

It may take a few seconds. The longer the dataset, the longer it will take.

In [ ]:
fuzzy_match(df_1, df_2, 'NAME_1', 'Nome_UF', threshold=95, limit=2)

,ID_1,NAME_1,geometry,matches
0,1,Acre,"MULTIPOLYGON (((-73.33251 -7.32488, -72.65935 ...",Acre
1,2,Alagoas,"MULTIPOLYGON (((-35.90153 -9.86180, -35.90097 ...",Alagoas
2,3,Amapá,"MULTIPOLYGON (((-50.02403 0.85986, -50.02431 0...",Amapá
3,4,Amazonas,"MULTIPOLYGON (((-67.32623 2.02968, -67.30034 1...",Amazonas
4,5,Bahia,"MULTIPOLYGON (((-38.69708 -17.97903, -38.69597...",Bahia
5,6,Ceará,"MULTIPOLYGON (((-38.47542 -3.70097, -38.47264 ...",Ceará
6,7,Distrito Federal,"MULTIPOLYGON (((-48.03603 -15.50022, -47.41730...",Distrito Federal
7,8,Espírito Santo,"MULTIPOLYGON (((-40.88403 -21.16125, -40.88208...",Espírito Santo
8,9,Goiás,"MULTIPOLYGON (((-50.15817 -12.41238, -50.16134...",Goiás
9,10,Maranhão,"MULTIPOLYGON (((-42.12375 -2.80069, -42.11653 ...",Maranhão


### 5.1 The matches

You can see there is now a new column in our .geojson dataframe called "matches." These are the matching names from our pcode dataset!

### 5.2 Bring over the pcodes

Now that we have matching columns we are know align, we can perform a table join the pull over the pcodes to the .geojson dataset.

In [ ]:
brazil_set_merged = pd.merge(brazil_set,
                             pcode_adm1[['Nome_UF','pcode_adm1']],
                             left_on='matches',
                             right_on='Nome_UF', 
                             how='left')

We can make sure it worked.

In [ ]:
brazil_set_merged.head()

,ID_1,NAME_1,geometry,matches,Nome_UF,pcode_adm1
0,1,Acre,"MULTIPOLYGON (((-73.33251 -7.32488, -72.65935 ...",Acre,Acre,BR12
1,2,Alagoas,"MULTIPOLYGON (((-35.90153 -9.86180, -35.90097 ...",Alagoas,Alagoas,BR27
2,3,Amapá,"MULTIPOLYGON (((-50.02403 0.85986, -50.02431 0...",Amapá,Amapá,BR16
3,4,Amazonas,"MULTIPOLYGON (((-67.32623 2.02968, -67.30034 1...",Amazonas,Amazonas,BR13
4,5,Bahia,"MULTIPOLYGON (((-38.69708 -17.97903, -38.69597...",Bahia,Bahia,BR29


## 6. Verifying the matches

We want to verify the matches. We can do a prelimninary search. This will give us a simple true or false. If it is false, we need to investigate more.

## 6.1 Method 1 - running a simple "equals" command

In [ ]:
brazil_set_merged['NAME_1'].equals(brazil_set_merged['matches'])

True

### 6.2 If you got "false" from the equals command

You'll need to more thoroughly search to find which rows are not equal.

In [ ]:
brazil_set_merged['match_check'] = (brazil_set_merged.NAME_1 == brazil_set_merged.matches) | True
brazil_set_merged['match_check'] = (brazil_set_merged.NAME_1 == brazil_set_merged.matches) | False

In [ ]:
# The "~" in the equation will return only false matches for us
match_check = brazil_set_merged[~brazil_set_merged.match_check]
print(match_check)

Empty GeoDataFrame
Columns: [ID_1, NAME_1, geometry, matches, Nome_UF, pcode_adm1, match_check]
Index: []


And it did! We can see the pcode_adm1 column at the end of the dataframe.

# Cleaning up the dataset

We are using this spatial dataset for HXL, so we want the filesize to be as small as possible. Because of this, we'll remove all columns we aren't using. We only want to keep the name, geometry, and pcode. 

Finally, we'll rename the columns to be used in HXL (admin names and pcode names must use these headers).

In [ ]:
#del brazil_set_merged['Nome_UF']
#del brazil_set_merged['matches']

In [ ]:
#del brazil_set_merged['ID_1']

In [ ]:
brazil_set_merged = brazil_set_merged.rename(columns={'NAME_1': 'admin1Name', 'pcode_adm1': 'admin1Pcode'})

In [ ]:
brazil_set_merged.head()

,admin1Name,geometry,admin1Pcode
0,Acre,"MULTIPOLYGON (((-73.33251 -7.32488, -72.65935 ...",BR12
1,Alagoas,"MULTIPOLYGON (((-35.90153 -9.86180, -35.90097 ...",BR27
2,Amapá,"MULTIPOLYGON (((-50.02403 0.85986, -50.02431 0...",BR16
3,Amazonas,"MULTIPOLYGON (((-67.32623 2.02968, -67.30034 1...",BR13
4,Bahia,"MULTIPOLYGON (((-38.69708 -17.97903, -38.69597...",BR29


In [ ]:
with open('test.geojson', 'w') as f:
    f.write(brazil_set_merged.to_json())